In [1]:
from pathlib import Path
import pickle

import open3d as o3
import numpy as np

from pointcloud.config import DATA_PATH
from pointcloud.utils.io import read_ply_file
from pointcloud.processors.sensat.plot import draw_pointcloud, draw_segmented_pointcloud

LABELS = {
    0: "Ground",
    1: "High Vegetation",
    2: "Buildings",
    3: "Walls",
    4: "Bridge",
    5: "Parking",
    6: "Rail",
    7: "traffic Roads",
    8: "Street Furniture",
    9: "Cars",
    10: "Footpath",
    11: "Bikes",
    12: "Water",
}

pointcloud_path = Path(DATA_PATH / "sensat_urban/grid_0.200/birmingham_block_11.ply")
kd_tree_path = Path(
    DATA_PATH / "sensat_urban/grid_0.200/birmingham_block_11_KDTree.pkl"
)
proj_path = Path(DATA_PATH / "sensat_urban/grid_0.200/birmingham_block_11_proj.pkl")

assert pointcloud_path.exists(), "PointCloud file not found."
assert kd_tree_path.exists(), "KDTree file not found."
assert proj_path.exists(), "Proj file not found."

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [11]:
data = read_ply_file(pointcloud_path)
point_colors = np.vstack((data["red"], data["green"], data["blue"])).T
point_labels = data["class"]
points = np.vstack((data["x"], data["y"], data["z"])).T

In [2]:
with open(proj_path, "rb") as kd_tree_file:
    proj = pickle.load(kd_tree_file)

In [4]:
LABEL_COLORS = [
    [85, 107, 47],  # ground -> OliveDrab
    [0, 255, 0],  # tree -> Green
    [255, 165, 0],  # building -> orange
    [41, 49, 101],  # Walls ->  darkblue
    [0, 0, 0],  # Bridge -> black
    [0, 0, 255],  # parking -> blue
    [255, 0, 255],  # rail -> Magenta
    [200, 200, 200],  # traffic Roads ->  grey
    [89, 47, 95],  # Street Furniture  ->  DimGray
    [255, 0, 0],  # cars -> red
    [255, 255, 0],  # Footpath  ->  deeppink
    [0, 255, 255],  # bikes -> cyan
    [0, 191, 255],  # water ->  skyblue
]

In [10]:
pointcloud_path = Path(DATA_PATH / "sensat_urban/train/cambridge_block_32.ply")

In [15]:
pointcloud = o3.geometry.PointCloud()
pointcloud.points = o3.utility.Vector3dVector(points)
pointcloud.colors = o3.utility.Vector3dVector(point_colors / 255.0)
o3.geometry.PointCloud.estimate_normals(pointcloud)

voxel_grid = o3.geometry.VoxelGrid.create_from_point_cloud(pointcloud, voxel_size=0.1)
o3.visualization.draw_geometries([voxel_grid])